In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
from scipy.stats import chi2_contingency

import requests
from bs4 import BeautifulSoup
import time

In [5]:
%run '0_tools.ipynb'

In [6]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

In [19]:
def scrape_yelp_reviews(txt, loc, business_type):
    
    counts = [0, 0, 0, 0, 0]
    
    city = loc[0]
    state = loc[1]
    
    city_web = ''
    for cw in city.split():
        city_web+=cw+'+'
    
    city_web = city_web[:len(city_web)-1]
    
    bus_web = ''
    for bw in business_type.split():
        bus_web+=bw+'+'
    
    bus_web = bus_web[:len(bus_web)-1]
    

    website = f'https://www.yelp.com/search?find_desc={bus_web}&find_loc={city_web}%2C+{state}&start=0'

    www = website
#     print(www)

    www = requests.get(www,headers = headers)
    if (www.status_code != 200):
        print("Error")

    soup = BeautifulSoup(www.text, 'html.parser')

    ul = (soup.find('ul', class_="undefined list__09f24__ynIEd"))

    lis = ul.find_all('li')

    links = set()
    for li in lis:
        
        try:
            h3 = li.find('h3')
            ahref = h3.find('a')
            link = ahref.get("href")
            if (link.find('/biz') == -1): continue
            link = f'https://yelp.com{link}'
            links.add(link)
            
        except:
            continue

    links = list(links)
    
    dataFile = open(txt, "a")
    
    for link in links:
        
        for target_rating in range(1, 5+1):
            
            www = link+f'&rr={target_rating}'
            
            www = requests.get(www,headers = headers)
            if (www.status_code != 200):
                print("Error")

            soup = BeautifulSoup(www.text, 'html.parser')
            str_soup = str(soup)
            reviews_index = (str_soup.find('"reviewRating":{"ratingValue":'))

            reviews = str_soup[reviews_index:]
            reviews_split = (reviews.split('}'))

            i=0
            
            while (i < len(reviews_split)):
                
                rating_index = reviews_split[i].find('reviewRating":{"ratingValue":')
                if (rating_index == -1): 
                    i+=1
                    continue

                rating = reviews_split[i][rating_index+29:]
                dataFile.write(rating + '\n - \n')

                i+=1
                
                review_text = reviews_split[i][15:]
                dataFile.write(review_text + '\n - \n - \n')

                i+=1

    dataFile.close()
            
#         print(f'Business {links.index(link)+1}/{len(links)} done')
        
    print(f'{city}, {state} {business_type} done')

In [8]:
def scrape_yellow_pages(txt, loc, business_type):    
    city = loc[0]
    state = loc[1]
    
    rating_dict = {'one':1, 'two':2, 'three':3, 'four':4, 'five':5}
    
    city_web = ''
    for cw in city.split():
        city_web+=cw+'+'
    
    city_web = city_web[:len(city_web)-1]
    
    bus_web = ''
    for bw in business_type.split():
        bus_web+=bw+'+'
        
    bus_web = bus_web[:len(bus_web)-1]
    
#     website = f'https://www.yellowpages.com/{city_web}{state}/{bus_web}'

    website = f'https://www.yellowpages.com/search?search_terms={bus_web}&geo_location_terms={city_web}%2C+{state}&s=average_rating'
        
    www = website

#     print(www)
    
    www = requests.get(www,headers = headers)
    
    if (www.status_code != 200):
        print(f"Error with {www}")
        return
        
        
    links = []
    soup = BeautifulSoup(www.text, 'html.parser')
    
    divs = (soup.find_all('div', 'info-section info-primary'))
    
    
    for div in divs:
        
        a = div.find('a', class_='rating')
        try:
            count = a.find('span', class_='count').text
            count = (int((count)[1:len(count)-1]))
            if (count > 1): 
                link = (f"https://www.yellowpages.com{a.get('href')}")
#                 link = link[:link.find('?')]
                links.append(link)
            
        except:
            continue
            
    dataFile = open(txt, "a")
    
    for url in links:
        
        www = url
        
        www = requests.get(www,headers = headers)
        if (www.status_code != 200):
            print("Error")
            
        soup = BeautifulSoup(www.text, 'html.parser')
        
        str_soup = str(soup)
        
#         print(str_soup.split('{"@type":"Rating","ratingValue":'))

        articles = soup.find_all('article', class_='clearfix')
        
        for r in articles:
            
            try:
                
                html_rating = r.find('div', class_= 'result-ratings overall').find('div')
                text = r.find('div', class_='review-response').text
                if (len(text) < 5): continue

                str_rating = html_rating.get('class')[1]
                rating = rating_dict[str_rating]
                
                dataFile.write(str(rating) + '\n - \n')
                dataFile.write(text + '\n - \n - \n')
                
            except:
                continue
                                
    dataFile.close()
    
    print(f'{loc[0]}, {loc[1]} {business_type} done')

In [9]:
business_type_dict = {0 : 'Accountants', 1 : 'Fast Food', 2 : 'Newspapers', 3 : 'Plumbers', 4 : 'Government', 
                      5:'Beaches', 6 : 'Law Firm', 7 :'Movie Theatre', 8: 'Pet Grooming', 9 : 'Real Estate'}
                      
location_dict = {0 : ['Seattle', 'WA'], 1 : ['Austin', 'TX'], 2 : ['Portland', 'OR'], 3 : ['Philadelphia', 'PA'], 
                4: ['Miami', 'FL'], 5 : ['Atlanta', 'GA'], 6 : ['Cincinnati', 'OH'], 7 : ['Dallas', 'TX'], 
                 8 : ['Detroit', 'MI'], 9 : ['San Francisco', 'CA']}

In [10]:
erase = False

In [20]:
txt_file = f'{root_path}data/yelp_reviews.txt'
if (erase): open(txt_file, 'w').close()
    
for l in range(len(location_dict)):
    for b in range(len(business_type_dict)):
        scrape_yelp_reviews(txt_file, location_dict[l], business_type_dict[b])

Business 1/9 done
Business 2/9 done
Business 3/9 done
Business 4/9 done
Business 5/9 done
Business 6/9 done
Business 7/9 done
Business 8/9 done
Business 9/9 done
New York, NY Accountants done
Business 1/10 done
Business 2/10 done
Business 3/10 done
Business 4/10 done
Business 5/10 done
Business 6/10 done
Business 7/10 done
Business 8/10 done
Business 9/10 done
Business 10/10 done
New York, NY Fast Food done
Business 1/10 done
Business 2/10 done
Business 3/10 done
Business 4/10 done
Business 5/10 done
Business 6/10 done
Business 7/10 done
Business 8/10 done
Business 9/10 done
Business 10/10 done
New York, NY Newspapers done
Business 1/10 done
Business 2/10 done
Business 3/10 done
Business 4/10 done
Business 5/10 done
Business 6/10 done
Business 7/10 done
Business 8/10 done
Business 9/10 done
Business 10/10 done
New York, NY Plumbers done
Business 1/10 done
Business 2/10 done
Business 3/10 done
Business 4/10 done
Business 5/10 done
Business 6/10 done
Business 7/10 done
Business 8/10 done

Business 9/10 done
Business 10/10 done
Chicago, IL Movie Theatre done
Business 1/10 done
Business 2/10 done
Business 3/10 done
Business 4/10 done
Business 5/10 done
Business 6/10 done
Business 7/10 done
Business 8/10 done
Business 9/10 done
Business 10/10 done
Chicago, IL Pet Grooming done
Business 1/10 done
Business 2/10 done
Business 3/10 done
Business 4/10 done
Business 5/10 done
Business 6/10 done
Business 7/10 done


KeyboardInterrupt: 

In [12]:
txt_file = f'{root_path}data/yellowpages.txt'
if (erase): open(txt_file, 'w').close()

for l in range(len(location_dict)):
    for b in range(len(business_type_dict)):
        scrape_yellow_pages(txt_file, location_dict[l], business_type_dict[b])

New York, NY Accountants done
New York, NY Fast Food done
New York, NY Newspapers done
New York, NY Plumbers done
New York, NY Government done
New York, NY Beaches done
New York, NY Law Firm done
New York, NY Movie Theatre done
New York, NY Pet Grooming done
New York, NY Real Estate done
Los Angeles, CA Accountants done
Los Angeles, CA Fast Food done
Los Angeles, CA Newspapers done
Los Angeles, CA Plumbers done
Los Angeles, CA Government done
Los Angeles, CA Beaches done
Los Angeles, CA Law Firm done
Los Angeles, CA Movie Theatre done
Los Angeles, CA Pet Grooming done
Los Angeles, CA Real Estate done
Boston, MA Accountants done
Boston, MA Fast Food done
Boston, MA Newspapers done
Boston, MA Plumbers done
Boston, MA Government done
Boston, MA Beaches done
Boston, MA Law Firm done
Boston, MA Movie Theatre done
Boston, MA Pet Grooming done
Boston, MA Real Estate done
Chicago, IL Accountants done
Chicago, IL Fast Food done
Chicago, IL Newspapers done
Chicago, IL Plumbers done
Chicago, IL G

In [49]:
def convert_kaggle_sentiments():
    
    txt_file = f'{root_path}data/kaggle_reddit.txt'
    open(txt_file, 'w').close()
    
    dataFile = open(txt_file, "a")

    df = pd.read_csv(f'{data_path}kaggle/Reddit_Data.csv')
    for index in range(len(df['clean_comment'])):
        sentiment = str(df['category'][index]+3)
        text = str(df['clean_comment'][index])
        text_len = len(text.split())
        
        if (text_len < 6): continue
            
        dataFile.write(sentiment + '\n - \n' + text + '\n - \n - \n')
    
    dataFile.close()
    
    txt_file = f'{root_path}data/kaggle_twitter.txt'
    open(txt_file, 'w').close()
    
    dataFile = open(txt_file, "a")

    df = pd.read_csv(f'{data_path}kaggle/Twitter_Data.csv')
    for index in range(len(df['clean_text'])):
        sentiment = str(df['category'][index]+3)
        text = str(df['clean_text'][index])
        text_len = len(text.split())
        
        if (text_len < 6): continue
            
        dataFile.write(sentiment + '\n - \n' + text + '\n - \n - \n')
    
    dataFile.close()
    

In [50]:
convert_kaggle_sentiments()